In [1]:
import pandas as pd
import os
from bs4 import BeautifulSoup as bs
import re
import itertools

In [2]:
fpath = 'rhyme_corpus'

filenames = [os.path.join(fpath,fname) for fname in os.listdir(fpath) if fname.endswith('.xml')]

In [3]:
line_lst = []
rhyme_lst = []
problem_lst = []

corr_lst = []
f_lst = []

cnt_p1 = 0
cnt_p2 = 0

for file in filenames:
    with open(file, "r",encoding="utf-8") as file:
        # Read each line in the file, readlines() returns a list of lines
        content = file.readlines()
        # Combine the lines in the list into a string
        content = "".join(content)
        bs_content = bs(content, "lxml")

    result = bs_content.find_all("lg")
    for item in result:
        bs_item = bs(str(item),'lxml')
        
        subitems = bs_item.find_all('lg')
        if len(subitems) <= 1:
            try:
                rhyme = re.search(r'rhyme=\"(.*?)\"', str(item)).group(1)
            except: 
                rhyme = 'z'
            lines = bs_item.find_all('l')

            for i in range(len(lines)):
                lines[i] = re.sub('<[^>]+>', '', str(lines[i]))
                lines[i] = re.sub('ſ', 's', str(lines[i]))
                lines[i] = re.sub('uͤ', 'ü', str(lines[i]))
                lines[i] = re.sub('aͤ', 'ä', str(lines[i]))
                lines[i] = re.sub('oͤ', 'ö', str(lines[i])).lower()
                lines[i] = re.sub(r'[^a-zäöü ]', '', str(lines[i]))

            lines = [line for line in lines if line] 
            rhyme_lst.append(rhyme)
            f_lst.append(file)
            
            if len(lines) != len(rhyme):
                lines_corr = []
                cnt = 0
                while cnt < len(lines): 
                    if cnt < len(lines)-1:
                        if len(lines[cnt+1].split()) < 3:
                            lines_corr.append(lines[cnt] + ' ' + lines[cnt + 1])
                            cnt += 2
                        else:
                            lines_corr.append(lines[cnt])
                            cnt +=1
                    else: 
                        lines_corr.append(lines[cnt])
                        cnt +=1                   
                
                
                if len(lines_corr) != len(rhyme): 
                    problem_lst.append(2)
                    cnt_p1 +=1
                    line_lst.append(lines)
                else: 
                    problem_lst.append(1)
                    line_lst.append(lines_corr)
            elif rhyme == 'z':
                problem_lst.append(2)
                cnt_p2 +=1
                line_lst.append(lines)
            else:
                problem_lst.append(0)
                line_lst.append(lines)
                
                
        else:
            pass


In [4]:
cnt_p1

408

In [5]:
rhyme_df = pd.DataFrame(list(zip(line_lst, rhyme_lst,problem_lst,f_lst)),
               columns =['text', 'rhyme','problem','file'])

In [6]:
rhyme_df.to_csv('rhyme_df.csv')

In [7]:
rhyme_df

,text,rhyme,problem,file
0,[ich fürcht es ist zu hoch doch besser hochges...,aabbcc,0,<_io.TextIOWrapper name='rhyme_corpus\\Abschat...
1,"[es höre der herr dein klagen in noth, es schü...",aabbccddee,0,<_io.TextIOWrapper name='rhyme_corpus\\Abschat...
2,[mit worten wird der feind vergebens angegriff...,aa,0,<_io.TextIOWrapper name='rhyme_corpus\\Abschat...
3,"[so steiget und fällt, die ehre das glücke der...",abca,2,<_io.TextIOWrapper name='rhyme_corpus\\Abschat...
4,"[rosen blühn auff deinen wangen, liljen führt ...",abba,0,<_io.TextIOWrapper name='rhyme_corpus\\Abschat...
...,...,...,...,...
9061,"[schöpfer hier ist dein geschöpfe, der gerings...",aabb,0,<_io.TextIOWrapper name='rhyme_corpus\\Zinzend...
9062,"[führst du meinen leibesschatten, rückwärts zu...",aabb,0,<_io.TextIOWrapper name='rhyme_corpus\\Zinzend...
9063,"[zeige mir nur deinen willen, der soll meine s...",aabb,0,<_io.TextIOWrapper name='rhyme_corpus\\Zinzend...
9064,"[sieger über tod und hölle, la die krancke lag...",aabb,0,<_io.TextIOWrapper name='rhyme_corpus\\Zinzend...


In [9]:
rhyme_pairs = []
no_rhyme_pairs = []

rhyme_df_clean = rhyme_df.drop(rhyme_df[rhyme_df['problem']>0].index)

for index, row in rhyme_df_clean.iterrows():
    
    scheme = row['rhyme']
    sent = row['text']
    new_pairs = []
    letters = set(scheme)
    for letter in letters:
        indices = [idx for idx, char in enumerate(scheme) if char == letter]
        
        if len(indices) >= 2: 
            new_pair = [sent[i].split()[-1] for i in indices]
            new_pairs.append(new_pair)
           
            if len(indices) > 2:
                rhyme_pairs += [list(item) for item in list(itertools.combinations(new_pair,2))]
            elif len(indices) == 2:
                rhyme_pairs.append(new_pair)

        else:
            #print(sent)
            #print(scheme)
            #print(indices)
            pass
        
    for idx in range(len(new_pairs)-1):                                          # ugly, but does the job fast
        try:
            no_rhyme_pairs.append([new_pairs[idx][0],new_pairs[idx+1][0]])
            no_rhyme_pairs.append([new_pairs[idx][1],new_pairs[idx+1][1]])
        except:
            pass
    

In [10]:
len(rhyme_pairs)

20482

In [11]:
rhyme_stat = [1]*len(rhyme_pairs) + [0]*len(no_rhyme_pairs)
pairs = rhyme_pairs + no_rhyme_pairs

for pair in pairs:
    
    if len(pair) < 1:
        print(pair)
word_1 = [pair[0] for pair in pairs]
word_2 = [pair[1] for pair in pairs]

rhyme_word_df = pd.DataFrame(list(zip(word_1, word_2,rhyme_stat)),
               columns =['word1', 'word2','rhyme'])

In [12]:
len(no_rhyme_pairs)

22156

In [13]:
rhyme_word_df

,word1,word2,rhyme
0,handen,schanden,1
1,wagen,schlagen,1
2,hochgestiegen,liegen,1
3,geschrey,sey,1
4,thron,zion,1
...,...,...,...
42633,leben,stillen,0
42634,beschwerden,hölle,0
42635,werden,lagerstelle,0
42636,quellen,kinde,0


In [14]:
shuffled_df = rhyme_word_df.sample(frac=1).reset_index(drop=True)


In [15]:
ratio = 0.8

df_train = shuffled_df[:int(len(shuffled_df)*ratio)]
df_val = shuffled_df[int(len(shuffled_df)*ratio):]

df_train.to_csv('data/train.csv',index=False)
df_val.to_csv('data/val.csv',index=False)

shuffled_df.to_csv('data/rhymes_clean.csv', index=False)

In [16]:
label_dict = {1:'y',0:'n'}

train_data_df = shuffled_df.copy()

train_data_df['rhyme'] = train_data_df['rhyme'].apply(lambda x: label_dict[x])

train_data_df.to_csv('data/rhymes_clean.tsv', index=False, sep='\t',header = None)


train_data_df

,word1,word2,rhyme
0,schier,ihr,y
1,liegt,fluges,n
2,speisen,korn,n
3,finsternü,begehen,n
4,gebrochen,gesund,n
...,...,...,...
42633,ein,abendschein,y
42634,begehrt,gewärt,y
42635,schrey,dingen,n
42636,freygebohren,erkohren,y
